In [ ]:
import torch
import torch.nn as NN
import os
import numpy as np

src_path = "E:\\MLiA_SourceCode\\machinelearninginaction\\Ch02\\"
# trainLen = len(os.listdir(src_path+"trainingDigits"))
# fr = open(src_path+"trainingDigits\\0_0.txt")
# for line in fr.readlines():
#     print(len(line))
#     break
# TestLen = len(os.listdir(src_path+"testDigits"))
class myCNN(NN.Module):
    def __init__(self):
        super(myCNN,self).__init__()
        self.conv1 = NN.Conv2d(1,16,5)
        self.conv2 = NN.Conv2d(16,32,5)
        self.pool = NN.MaxPool2d(2,2)
        self.fc1 = NN.Linear(32*5*5,100)
        self.fc2 = NN.Linear(100,10)
        self.softmax = NN.Softmax(dim=1)
        self.activation = NN.ReLU()
        
    def forward(self,x):
        x = self.conv1(x) ## 16*28*28
        x = self.pool(x) ## 16*14*14
        x = self.conv2(x) ## 32*10*10
        x = self.pool(x) ## 32*5*5
        x = x.view(-1, 32*5*5) ##flatten
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.softmax(x)
        return x

train_label = []
train_labels = []
one_hot_tmp= np.zeros((1,10),dtype=np.int16)
for labelArr in (os.listdir(src_path+"trainingDigits")):
    trueLabel = int(labelArr.strip().split("_")[0])
    train_labels.append(trueLabel)
    one_hot_tmp[0][trueLabel] = 1
    train_label.append(one_hot_tmp.tolist())
#     one_hot_tmp[trueLabel] = 0
    one_hot_tmp= np.zeros((1,10),dtype=np.int16)
net = myCNN()
# print(train_label)
def getDataMatrix(path):
    flist = os.listdir(path)
    DataMatrix = []
    for i in flist:
        tmpMat = np.zeros((32,32))
        fr = open(path+'\\'+i)
        tmp = np.zeros((32,32),dtype=np.int16)
        lines = fr.readlines()
        length =len(lines)
        for j in range(length):
             for k in range(len(lines[j])-1):
                 tmpMat[j,k]= lines[j][k]
            
        DataMatrix.append(tmpMat.tolist())
    return DataMatrix
        
trainMat = getDataMatrix(src_path+"trainingDigits")
trainMat = torch.tensor(trainMat,dtype=torch.float32).unsqueeze(1)
train_labels = torch.tensor(train_labels,dtype=torch.long)
train_label = torch.tensor(train_label,dtype=torch.float32).squeeze(1)
print(trainMat.size(),train_label.size(),train_labels.size())

In [59]:
# x1 = torch.rand(1,1,32,32)
# print(x1)
import torch.optim as optim
loss_fn = NN.MSELoss()
optimizier = optim.SGD(net.parameters(),lr=1e-4)
# optimizier = optim.Adam(net.parameters(),lr = 1e-6)

# miniBatchSize = 20
# round = int(500/miniBatchSize) 
for i in range(500):
#     out = net(trainMat[i*miniBatchSize:i*miniBatchSize+19,:])
#     loss = loss_fn(out, train_label[i*miniBatchSize:i*miniBatchSize+19])
    out = net(trainMat)
    loss = loss_fn(out, train_label)
    optimizier.zero_grad()
    loss.backward()
    optimizier.step()
# print(out[1])

In [60]:
ec = 0
Error_dic = {}
Error_index = []
for i in range(len(train_labels)):
    if torch.argmax(out[i],dim=0).item()!=train_labels[i]:
        ec = ec + 1
        Error_index.append(i)
        Error_dic[train_labels[i].item()] = Error_dic.get(train_labels[i].item(),0)+1
# print(torch.argmax(out[i],dim=0).item(),train_labels[i])
print(Error_dic)
print(ec, ec/len(train_label))

{0: 189, 1: 1, 3: 1, 4: 2, 6: 2, 9: 1}
196 0.10134436401240951


In [61]:
PATH = './minstCnnByMyOwn_net.pth'
torch.save(net.state_dict(), PATH)
net = myCNN()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [62]:
import numpy as np
testMat = getDataMatrix(src_path+"testDigits")
testMat = torch.tensor(testMat,dtype=torch.float32).unsqueeze(1)
test_label = []
for labelArr in (os.listdir(src_path+"testDigits")):
    trueLabel = int(labelArr.strip().split("_")[0])
    test_label.append(trueLabel)
out = net(testMat)
test_error_dict={} 
ec_test = 0
for i in range(len(test_label)):
#     print(torch.argmax(out[i].item(),dim=0), test_label[i])
    if (torch.argmax(out[i],dim=0).item())!= test_label[i]:
        test_error_dict[test_label[i]] = test_error_dict.get(test_label[i], 0)+1
        ec_test = ec_test +1

print(test_error_dict,ec_test/len(test_label))
# print(“error rate is {}”.format{test_error_dict.values})

{0: 87, 1: 1, 2: 2, 3: 6, 4: 1, 5: 3, 6: 1, 7: 2, 8: 1, 9: 2} 0.11205073995771671
